#LiveCon Printer
This is a proof of concept, to directly producting a Program book,
out of the LiveCon CSV.


In [1]:
#CONFIG
import pytz
LOCAL_TIMEZONE = pytz.timezone("Australia/Perth")
SLOT_LENGTH_MINUTES = 30


In [2]:
def normalise(string): 
    import unicodedata
    import pylatex.utils
    import re
    string = string.replace("\r",'')
    string = str(pylatex.utils.escape_latex(string))
    string = re.sub(r'(\\\\)', r'\\', string )
    return string.strip()
    

def load_list(csl):
    ldlst =  str.split(normalise(csl), ', ')
    if ldlst==['']:
        return []
    else:
        return ldlst
    
def parse_datetime(datetime_str):
    import dateutil.parser
    
    raw = dateutil.parser.parse(datetime_str)
    return raw.astimezone(LOCAL_TIMEZONE)


class session (object):
    def __init__(self, id,
                 start_time_str,end_time_str,
                 title,tags_str,people_str,
                 venues_str,description):
        self.id = int(id)
        self.start = parse_datetime(start_time_str)
        self.end = parse_datetime(end_time_str)
        self.title = normalise(title)
        self.tags = load_list(tags_str)
        self.people = load_list(people_str)
        self.venues = load_list(venues_str)
        self.description = normalise(description)
    
    @property
    def day(self):
        week_days   = ['Sunday', 
          'Monday', 
          'Tuesday', 
          'Wednesday', 
          'Thursday',  
          'Friday', 
          'Saturday']
        daynum = self.start.weekday()
        return week_days[daynum]
    
    
    @property
    def start_time(self):
        return self.start.strftime("%H:%M")
    
    @property
    def end_time(self):
        return self.end.strftime("%H:%M")
    

In [3]:
import csv

sessions = []
with open('con_data/swacon_draft0_no15min_2015.csv', 'rU') as csvfile:
    con_csv = csv.reader(csvfile)
    next(con_csv) #Skip heading
    sessions = [session(*row) for row in con_csv]

sessions.sort(key=lambda ss: ss.start)



In [6]:
def write_descriptions(sessions, doc):
    from pagelayout import Multicols
    from itertools import groupby
    with doc.create(Section('Program',numbering=False)):
        with doc.create(Multicols(2)):
            for day_name, day_session in groupby(sessions, lambda ss: ss.day):
                with doc.create(Subsection(day_name,numbering=False)):
                    for session in day_session:
                        with doc.create(Subsubsection(session.title,numbering=False)):
                            with doc.create(Description()) as desc:
                                doc.append('\setlength{\itemsep}{0pt}')
                                doc.append('\setlength{\parsep}{0pt}')
                                doc.append('\setlength{\parskip}{0pt}')
                                desc.additem("Time:", session.start_time+" -- "+session.end_time)

                                if len(session.venues)>0:
                                    desc.additem("Venue:", ", ".join(session.venues))
                                if len(session.people)>0:
                                    desc.additem("People:", ", ".join(session.people))
                                if len(session.tags)>0:
                                    desc.additem("Tags:", ", ".join(session.tags))
                            doc.append(session.description)

In [5]:
import pylatex
from pylatex import Document, Section, Subsection, Subsubsection, Table, Package
from pylatex.command import Command
from lists import Description

from tabulartimetable import write_timetable


doc = Document(documentclass="scrreprt")
doc.packages.append(Package('geometry', options=['tmargin=1cm','bmargin=1cm',
                                                 'lmargin=1cm','rmargin=1cm',]))

write_timetable(sessions,doc, SLOT_LENGTH_MINUTES)
write_descriptions(sessions,doc)


NameError: name 'Multicols' is not defined

In [ ]:
from IPython.display import FileLink, FileLinks
with open("out/temp.tex", 'w') as temp_out:
    doc.dump(temp_out)

!lualatex --output-directory=out --interaction=nonstopmode temp.tex
FileLinks("./out")

In [ ]:
FileLinks("./out")